In [1]:
import json
from quam.components import *

from pathlib import Path
root_folder = Path("./output")
root_folder.mkdir(exist_ok=True)
for key in ["basic", "referenced", "referenced_multifile"]:
    (root_folder / key).mkdir(exist_ok=True)

2023-09-25 13:34:12,923 - qm - INFO     - Starting session: abf92272-c7ae-49af-bb0b-50e8c77a4011


# Creating QuAM from scratch

The user defines his own QuAM by inheriting from `QuamRoot` (see `components/quam.py`).
Once defined, it can be instantiated as follows:

In [2]:
quam = QuAM()
quam

QuAM(mixers=[], qubits=[], resonators=[], local_oscillators=[], analog_inputs=[], wiring=QuamDict())

We see that five attributes are defined in quam, all empty so far. We can begin populating it with different `QuamComponent`s

In [3]:
num_qubits = 2
for idx in range(num_qubits):
    # Create qubit components
    local_oscillator_qubit = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_qubit)

    mixer_qubit = Mixer(
        id=f"mixer_q{idx}",
        local_oscillator=local_oscillator_qubit,
        port_I=1,
        port_Q=2,
        frequency_drive=5e9,
    )

    transmon = Transmon(
        id=idx,
        xy=IQChannel(
            mixer=mixer_qubit
        ),
        z=SingleChannel(port=5),
    )
    quam.qubits.append(transmon)

    # Create resonator components
    local_oscillator_resonator = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_resonator)

    resonator_mixer = Mixer(
        id=f"mixer_r{idx}",
        local_oscillator=local_oscillator_resonator,
        port_I=3,
        port_Q=4,
        frequency_drive=5e9,
    )

    readout_resonator = ReadoutResonator(id=idx, mixer=resonator_mixer)
    quam.resonators.append(readout_resonator)

# Create analog inputs
quam.analog_inputs.append(AnalogInput(port=1))
quam.analog_inputs.append(AnalogInput(port=2))

quam

QuAM(mixers=[], qubits=[Transmon(id=0, frequency=None, xy=IQChannel(pulses=QuamDict(), mixer=Mixer(id='mixer_q0', local_oscillator=LocalOscillator(power=10, frequency=6000000000.0), port_I=1, port_Q=2, frequency_drive=5000000000.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0, controller='con1')), z=SingleChannel(pulses=QuamDict(), port=5, filter_fir_taps=None, filter_iir_taps=None, offset=0, controller='con1')), Transmon(id=1, frequency=None, xy=IQChannel(pulses=QuamDict(), mixer=Mixer(id='mixer_q1', local_oscillator=LocalOscillator(power=10, frequency=6000000000.0), port_I=1, port_Q=2, frequency_drive=5000000000.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0, controller='con1')), z=SingleChannel(pulses=QuamDict(), port=5, filter_fir_taps=None, filter_iir_taps=None, offset=0, controller='con1'))], resonators=[ReadoutResonator(pulses=QuamDict(), id=0, mixer=Mixer(id='mixer_r0', local_oscillator=LocalOscillator(power=10, frequency=6000000000.0), port_I

## Saving and loading quam

We can then save quam to a json file:

In [4]:
quam.save(root_folder / "basic" / "state.json")

Finally, we can also load quam from the same json file

In [5]:
loaded_quam = QuAM.load(root_folder / "basic" / "state.json")

Importantly, the `state.json` file only describes how QuAM is structured, and how the different QuAM components should be initialized. The components themselves are defined in their respective classes.

## Generating QUA config

We can also generate the qua config from quam. This recursively calls `QuamComponent.apply_to_config()` on all quam components.

In [6]:
qua_config = quam.generate_config()
json.dump(qua_config, open(root_folder / "basic" / "qua_config.json", "w"), indent=2)

In [7]:
root_folder / "basic" / "qua_config.json"

PosixPath('output/basic/qua_config.json')

# Quam using references

QuAM allows values to reference another part of QuAM by setting its value to a string starting with a colon: `":path.to.referenced.value"`.

As an example why this is useful, we previously hardcoded all output ports. However, grouping everything at a top-level `"wiring"`` makes more sense, so we can have all output ports reference to it.

Below we also add several more references

In [8]:
num_qubits = 2
quam = QuAM()
quam.wiring = {
    "qubits": [
        {"port_I": 5 * k + 1, "port_Q": 5 * k + 2, "port_Z": 5 * k + 3}
        for k in range(num_qubits)
    ],
    "resonators": [
        {"port_I": 5 * k + 4, "port_Q": 5 * k + 5} for k in range(num_qubits)
    ],
}

for idx in range(num_qubits):
    # Create qubit components
    local_oscillator_qubit = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_qubit)

    mixer_qubit = Mixer(
        id=f"mixer_q{idx}",
        local_oscillator=f":local_oscillators[{idx}]",
        port_I=f":wiring.qubits[{idx}].port_I",
        port_Q=f":wiring.qubits[{idx}].port_Q",
        frequency_drive=f":qubits[{idx}].frequency",
    )
    quam.mixers.append(mixer_qubit)

    transmon = Transmon(
        id=idx,
        xy=IQChannel(mixer=f":mixers[{2*idx}]"),
        z=SingleChannel(port=f":wiring.qubits[{idx}].port_Z"),
        frequency=6.1e9,
    )
    quam.qubits.append(transmon)

    # Create resonator components
    local_oscillator_resonator = LocalOscillator(power=10, frequency=6e9)
    quam.local_oscillators.append(local_oscillator_resonator)

    resonator_mixer = Mixer(
        id=f"mixer_r{idx}",
        local_oscillator=f":local_oscillators[{idx}]",
        port_I=f":wiring.resonators[{idx}].port_I",
        port_Q=f":wiring.resonators[{idx}].port_Q",
        frequency_drive=f":resonators[{idx}].frequency",
    )
    quam.mixers.append(resonator_mixer)

    readout_resonator = ReadoutResonator(
        id=idx,
        mixer=f":mixers[{2*idx+1}]",
        frequency=5.9e9,
    )
    quam.resonators.append(readout_resonator)

# Create analog inputs
quam.analog_inputs.append(AnalogInput(port=1))
quam.analog_inputs.append(AnalogInput(port=2))

We again save quam and the QUA config. The QUA config is identical to previous, but QuAM has changed significantly

In [ ]:
quam.save(root_folder / "referenced" / "state.json")

In [ ]:
qua_config = quam.generate_config()
json.dump(qua_config, open(root_folder / "referenced" / "qua_config.json", "w"), indent=2)

## Separating QuAM into multiple files

Finally, we can also separate parts of QuAM to be placed into a separate file.
This can be especially useful when combined with referencing because we can now have a separate file dedicated to the wiring of the experiment.

Here we point quam to a folder instead of a json file, and we specify that `"wiring"` should go to a separate `wiring.json` file in that folder:

In [ ]:
quam.save(root_folder / "referenced_multifile" / "quam", content_mapping={"wiring.json": ["wiring"]})

It can subseuently be loaded as usual

In [ ]:
quam = QuAM.load(root_folder / "referenced_multifile" / "quam")

Separating QuAM into multiple files allows the user to revert back to a previous QuAM but keep some parts the same.
For example, the user may want to revert all experimental settings, but the wiring of the setup should of course not change.